# Exercise: Exact quantum dyanmics for 1D 

In [ ]:
from widget_code_input import WidgetCodeInput

code_widget = WidgetCodeInput(
    function_name = "generate_potential_file",
    function_parameters = "xmin, xmax, num",
    docstring="""
    A function to write the potential file for the SOFT program. 
    
    :xmin is the left range of the potential.
    :xmax is the right range of the potential.
    :num is the number of the points. 
    """,
    
    function_body="# Input here your solution\n"

)

display(code_widget)

## Run the SOFT program

1. Box potential.
2. Morse potential.
3. Harmonic potential.

In [15]:
import ipywidgets as widgets
import os
import subprocess
from time import *
from ipywidgets import HBox, VBox, HTML, FloatSlider, IntSlider, IntProgress
import shlex
from IPython.display import Image
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from IPython.display import display, clear_output
from IPython.display import Image
import time
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler


class MyHandler(FileSystemEventHandler):
    def on_modified(self, event):
        fprogress = open("./progress/count.dat");
        progress_bar.value = int(fprogress.readlines()[0].split()[0])
        fprogress.close()
        



event_handler = MyHandler()
observer = Observer()
observer.schedule(event_handler, path='./progress/', recursive=False)
observer.start()


out = widgets.Output()
out2 = widgets.Output()

def on_savefig(b):
    fig1 = plt.figure(figsize=(14,6))

    fpot = open("potential.dat","r");
    lines = fpot.readlines();

    x1 = [];
    y1 = [];

    for line in lines:
        if len(line.split()) == 3:
            a = line.split()[0]
            b = line.split()[1]
            c = line.split()[2]
            x1.append(float(b))
            y1.append(float(c)/10.0)

    plt.plot(x1, y1, 'green',label='potential')

    fpsi = open("final_psi.dat", "r");
    lines = fpsi.readlines();

    a1 = [];
    b1 = [];
    c1 = [];
    d1 = [];

    for line in lines:
        a = line.split()[1]
        b = line.split()[2]
        c = line.split()[3]
        d = line.split()[4]
        a1.append(float(a))
        b1.append(float(b))
        c1.append(float(c))
        d1.append(float(d))

    plt.plot(a1, b1, 'b', label="Psi Real");
    plt.plot(a1, c1, 'r', label="Psi Image"); 
    plt.plot(a1, d1, 'k', label="Psi Square");
    plt.xlim([0, 50])
    plt.ylim([-0.6, 0.6])


    plt.grid()
    plt.legend(fontsize=20)
    plt.savefig("test.pdf")


def run_soft(pot_type, dt, mass, pt, bh, bw):
    command = "./SOFT.x " + str(pot_type) + ' ' + str(dt);
    command = command + ' ' + str(mass) + ' ' + str(pt) + ' ' + str(bh) + ' ' + str(bw);
    p = subprocess.check_call(command, shell=True)
    p = subprocess.check_call('tail -512 psi.dat > final_psi.dat', shell=True)
    p = subprocess.check_call('./make_gif.sh', shell=True)
    
    fpot = open("potential.dat","r");
    lines = fpot.readlines();

    x1 = [];
    y1 = [];

    for line in lines:
        if len(line.split()) == 3:
            a = line.split()[0]
            b = line.split()[1]
            c = line.split()[2]
            x1.append(float(b))
            y1.append(float(c)/10.0)

    fpsi = open("final_psi.dat", "r");
    lines = fpsi.readlines();

    a1 = [];
    b1 = [];
    c1 = [];
    d1 = [];

    for line in lines:
        a = line.split()[1]
        b = line.split()[2]
        c = line.split()[3]
        d = line.split()[4]
        a1.append(float(a))
        b1.append(float(b))
        c1.append(float(c))
        d1.append(float(d))    
    
    fpot.close()
    fpsi.close()
    
    return x1, y1, a1, b1, c1, d1
    
            
def show_animation(b):
    with out2:
        out2.clear_output()
        display(Image(filename='./result.gif'))
        
def run_soft_button(b):
    run_button.style.button_color = 'red'
    out2.clear_output()
    x1, y1, a1, b1, c1, d1 = run_soft(w.value, dt.value, mass.value, pt.value, BH.value, BW.value)
    run_button.style.button_color = 'green'
    with out:
        out.clear_output(wait=True)
        fig = plt.figure(figsize=(14,6))
        
        plt.plot(x1, y1, 'green',label='potential')
        plt.plot(a1, b1, 'b', label="$\Psi$ Real");
        plt.plot(a1, c1, 'r', label="$\Psi$ Image"); 
        plt.plot(a1, d1, 'k', label="$\Psi^2$");
        plt.xlim([0, 50])
        plt.ylim([-0.6, 0.6])
        
        
        plt.grid()
        plt.legend(fontsize=20)
        plt.show()
    
w = widgets.Dropdown(
    options=['None', '1', '2', '3'],
    value='None',
    description='Potential type:',
    disabled=False,
)

def on_value_change(b):
    if w.value == '1':
        BH.layout = layout_visible
        BW.layout = layout_visible
    else:
        BH.layout = layout_hidden
        BW.layout = layout_hidden

run_button = widgets.Button(description='Run SOFT')
save_button = widgets.Button(description='Save figure')
show_button = widgets.Button(description='Show animation')
progress_bar = IntProgress(value=0, min=0, max=100, bar_style='success')
save_button.on_click(on_savefig)
show_button.on_click(show_animation)
run_button.on_click(run_soft_button)

layout_hidden  = widgets.Layout(visibility = 'hidden')
layout_visible = widgets.Layout(visibility = 'visible')


BH = IntSlider(value=0, min=0, max=10, description='BH');
BW = IntSlider(value=0, min=0, max=10, description='BW');

BH.layout = layout_hidden
BW.layout = layout_hidden
        
dt = FloatSlider(value=0.01, min=0.01, max=0.10, step=0.01, description='dt')
mass = IntSlider(value=1, min=1, max=10, description='mass')
pt = IntSlider(value=0, min=0, max=10, description='Pause time per step')
# dt.observe(on_value_change, names='value', type='change')
w.observe(on_value_change, names='value', type='change')
display(VBox([w, BH, BW, dt, mass, pt, HBox([run_button, progress_bar]), HBox([save_button, show_button]),out2]))